In [2]:
import pandas as pd
from statistics import *
from math import isnan
import matplotlib.pyplot as plt # библиотека визуализации
from scipy import stats # библиотека для расчетов
from sklearn.metrics import matthews_corrcoef 
import seaborn as sns
import plotly
import plotly.express as px

В футболе статистические функции могут представлять собой хорошо известные модели, такие как модель регрессии Пуассона или модель Диксона и Коулза.

In [3]:
season = {
    "bundes_22-23":[ 1736824, 1737122,298 ],
    "Prem_Leag_22-23":[1736219,1736589,370],
    "Premiere_22-23":[1744303,1744668,365],
    "Series_A_22-23":[1747394,1747768,374]
}

In [22]:
df_b = pd.read_csv("data/bundes_22-23.csv")
df_l = pd.read_csv("data/table_res_math_Prem_Leag_22-23.csv")
df = pd.concat([df_b,df_l])
df = df.drop('Unnamed: 0',axis=1)

In [23]:
display(df.head())
df.info()

,id,общая информация,Дата,Команда1,счет1,Команда2,счет2,Температура,Признак погоды,поб1,...,Офсайды (соперник) 1,Офсайды (соперник) 2,Желтые карточки 1,Желтые карточки 2,Желтые карточки (соперник) 1,Желтые карточки (соперник) 2,Красные карточки 1,Красные карточки 2,Красные карточки (соперник) 1,Красные карточки (соперник) 2
0,1736825,"Бундеслига, 1-й тур, регулярный сезон, 05.08.2...",05.08.2022 21:35,Айнтрахт Франкфурт,1,Бавария,6,+29°C,солнечно,6.50,...,2.00,2.5,0.88,1.90,1.00,1.50,0.0,0.10,0.25,0.0
1,1736826,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",06.08.2022 19:30,Боруссия Дортмунд,1,Байер,0,+27°C,солнечно,2.13,...,1.63,1.0,0.75,0.89,1.25,1.00,0.0,0.00,0.00,0.0
2,1736827,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",06.08.2022 16:30,Унион Берлин,3,Герта,1,+28°C,солнечно,1.87,...,0.67,2.0,1.20,1.75,0.80,0.75,0.0,0.25,0.00,0.0
3,1736828,"Бундеслига, 1-й тур, регулярный сезон, 07.08.2...",07.08.2022 18:30,Кёльн,3,Шальке,1,+30°C,солнечно,2.08,...,1.40,2.5,2.00,0.40,1.67,0.00,0.0,0.00,0.00,0.0
4,1736829,"Бундеслига, 1-й тур, регулярный сезон, 06.08.2...",06.08.2022 16:30,Боруссия М,3,Хоффенхайм,1,+27°C,солнечно,2.07,...,2.33,2.0,0.29,2.00,0.57,1.40,0.0,0.00,0.00,0.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 622 entries, 0 to 323
Data columns (total 72 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              622 non-null    int64  
 1   общая информация                622 non-null    object 
 2   Дата                            622 non-null    object 
 3   Команда1                        622 non-null    object 
 4   счет1                           622 non-null    object 
 5   Команда2                        622 non-null    object 
 6   счет2                           622 non-null    object 
 7   Температура                     622 non-null    object 
 8   Признак погоды                  622 non-null    object 
 9   поб1                            622 non-null    float64
 10  X                               622 non-null    float64
 11  поб2                            622 non-null    float64
 12  ТМ 2.5                          622 

In [32]:
df["счет1"]

1    105
1     84
2     82
2     76
0     72
0     52
3     39
3     39
4     22
4     19
5     16
6      4
6      4
5      4
-      2
9      1
7      1
Name: счет1, dtype: int64

In [33]:
df["тотал"] = df["счет1"] , df["счет2"]

ValueError: Length of values (2) does not match length of index (622)

In [ ]:
fig = px.imshow(
    df.corr(),
    text_auto=True,
    height=1400,
    width=1400
)
fig.show()

In [ ]:
df[["Температура","Признак погоды"]].value_counts()

Температура  Признак погоды                                          
+17°C        небольшой дождь                                             97
+27°C        солнечно                                                    71
+17°C        переменная облачность                                       65
+28°C        солнечно                                                    65
+29°C        солнечно                                                    51
+28°C        переменная облачность                                       34
+17°C        пасмурно                                                    33
+30°C        солнечно                                                    31
+17°C        умеренный или сильный ливневый дождь                        19
+29°C        умеренный или сильный ливневый дождь                        17
+16°C        переменная облачность                                       17
+18°C        умеренный дождь                                             16
+19°C        перем

In [ ]:
df['Признак погоды'].value_counts()

солнечно                                                    248
переменная облачность                                       148
небольшой дождь                                              97
пасмурно                                                     49
умеренный или сильный ливневый дождь                         36
умеренный дождь                                              16
в отдельных районах умеренный или сильный дождь с грозой     15
временами умеренный дождь                                    13
Name: Признак погоды, dtype: int64

In [ ]:
df[df["счет1"] == '-']["id"]

136    1736961
288    1737113
Name: id, dtype: int64

In [ ]:
display(df[df['id'] == 1736961])
display(df[df['id'] == 1737113])

,id,общая информация,Дата,Команда1,счет1,Команда2,счет2,Температура,Признак погоды,поб1,...,Офсайды (соперник) 1,Офсайды (соперник) 2,Желтые карточки 1,Желтые карточки 2,Желтые карточки (соперник) 1,Желтые карточки (соперник) 2,Красные карточки 1,Красные карточки 2,Красные карточки (соперник) 1,Красные карточки (соперник) 2
136,1736961,"Суперкубок, финал, 12.08.2023 21:45",12.08.2023 21:45,Бавария,-,РБ Лейпциг,-,+28°C,солнечно,1.72,...,0.8,1.2,1.5,2.4,1.9,2.6,0.0,0.2,0.0,0.0


,id,общая информация,Дата,Команда1,счет1,Команда2,счет2,Температура,Признак погоды,поб1,...,Офсайды (соперник) 1,Офсайды (соперник) 2,Желтые карточки 1,Желтые карточки 2,Желтые карточки (соперник) 1,Желтые карточки (соперник) 2,Красные карточки 1,Красные карточки 2,Красные карточки (соперник) 1,Красные карточки (соперник) 2
288,1737113,"Суперкубок, финал, 12.08.2023 21:45",12.08.2023 21:45,Бавария,-,РБ Лейпциг,-,+28°C,солнечно,1.72,...,0.8,1.2,1.5,2.4,1.9,2.6,0.0,0.2,0.0,0.0
